In [19]:
class FrozenLake():
    def __init__(self) -> None:
        super(FrozenLake, self).__init__()
        self.reset_coo()
        self.actions = [
            [-1, 0], # Up
            [1, 0], #Down
            [0, -1], # Left
            [0, 1] # Right
        ]

    def reset_coo(self):
        self.x = 0
        self.y = 0
        self.score = 0
        self.grid = [[0,0,0,-1], [0,0,-1,-1], [-1,0,0,0], [-1,0,0,1]]
        return 0
    
    def show(self):
        r = "________________________________________\n"
        y = 0
        for line in self.grid:
            x = 0
            for pt in line:
                if y != self.y or x != self.x:
                    r += str(pt)+"\t"
                else :
                    r += "X\t"
                # print("%s\t" % (pt if y != self.y or x != self.x else "X"), end="")
                x += 1
            y += 1
            r += "\n"
        return r

    def is_finished(self):
        return self.grid[self.y][self.x] == 1
    
    def step(self, action):
        """
            Action: 0, 1, 2, 3
        """
        self.y = max(0, min(self.y + self.actions[action][0],len(self.grid)-1))
        self.x = max(0, min(self.x + self.actions[action][1],len(self.grid)-1))

        return (self.y*len(self.grid[0])+self.x) , self.grid[self.y][self.x]

In [24]:
from random import randint
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

def take_action(st, Q, eps):
    # Take an action
    if randint(0, 100)/100 < eps:
        action = randint(0, 3)
    else: # Or greedy action
        action = np.argmax(Q[st])
    return action

def showQ(Q, minmaxscaler=False):
    q = pd.DataFrame(Q, columns=["0(haut)", "1(bas)", "2(gauche)", "3(droite)"])
    if not minmaxscaler:
        return q
    q = pd.DataFrame(MinMaxScaler().fit_transform(q.values), columns=q.columns)
    for y in range(len(Q)):
        for x in range(len(Q[y])):
            if max(Q[y]) - min(Q[y]) == 0:
                q.iloc[y][x] = 0
            else:
                q.iloc[y][x] = (Q[y][x] - min(Q[y])) / (max(Q[y]) - min(Q[y]))
    return q

In [26]:
jeu = FrozenLake()

Q = [[0, 0, 0, 0] for _ in range(len(jeu.grid)*len(jeu.grid[0]))]
eps = 0.9
gamma = 0.1
print(jeu.show())
for i in range(1000):
    lenght_path = 0
    st = jeu.reset_coo()
    while not jeu.is_finished():
        a = take_action(st, Q, eps)
        stp1, r = jeu.step(a)
        ap1 = take_action(st, Q, eps)
        malus =  - min((0.015*lenght_path)**1.5, 5) if jeu.y in [0,1,2,3] and jeu.x in [0,1,2,3] else -0.1 - min((0.015*lenght_path)**1.5, 5)
        Q[st][a] = Q[st][a] + gamma*(r + Q[stp1][ap1]-Q[st][a])  + malus
        st = stp1
        lenght_path += 1
showQ(Q)

________________________________________
X	0	0	-1	
0	0	-1	-1	
-1	0	0	0	
-1	0	0	1	



,0(haut),1(bas),2(gauche),3(droite)
0,-102.242442,-99.818833,-102.668754,-96.642668
1,-98.370878,-94.368849,-101.833037,-93.858956
2,-92.298240,-89.393306,-97.536333,-91.630900
3,-92.741838,-85.290455,-92.075358,-91.111434
4,-105.232201,-96.515072,-98.650854,-96.807988
5,-100.018716,-86.988392,-99.755943,-88.123703
6,-94.513660,-67.769858,-96.179020,-83.920554
7,-93.521661,-55.660284,-87.925441,-89.645959
8,-101.665915,-85.922655,-91.278493,-88.547962
9,-97.410694,-73.280716,-98.773030,-71.235727


In [18]:
jeu = FrozenLake()
st = jeu.reset_coo()
aL = []

while not jeu.is_finished():
    print(jeu.show())
    a = take_action(st, Q, 0.01)
    stp1, r = jeu.step(a)
    print(r)
    aL.append(r)
    st = stp1
print(jeu.show())

print(sum(aL), len(aL), aL)

________________________________________
X	0	0	-1	
0	0	-1	-1	
-1	0	0	0	
-1	0	0	1	

0
________________________________________
0	X	0	-1	
0	0	-1	-1	
-1	0	0	0	
-1	0	0	1	

0
________________________________________
0	0	0	-1	
0	X	-1	-1	
-1	0	0	0	
-1	0	0	1	

0
________________________________________
0	0	0	-1	
0	0	-1	-1	
-1	X	0	0	
-1	0	0	1	

0
________________________________________
0	0	0	-1	
0	0	-1	-1	
-1	0	X	0	
-1	0	0	1	

0
________________________________________
0	0	0	-1	
0	0	-1	-1	
-1	0	0	X	
-1	0	0	1	

1
________________________________________
0	0	0	-1	
0	0	-1	-1	
-1	0	0	0	
-1	0	0	X	

1 6 [0, 0, 0, 0, 0, 1]
